In [17]:
from confluent_kafka import Consumer, KafkaException, KafkaError
import avro.schema
import avro.io
import io
import sys

In [18]:
cfg = {
    "bootstrap.servers": "kafka:9092",
    "group.id": "group-t1",
    "default.topic.config": {"auto.offset.reset": "earliest"}}

In [19]:
consumer = Consumer(**cfg)
consumer.subscribe(['t1'])

In [20]:
def load_scheme(scheme_file):
    return avro.schema.Parse(open(scheme_file).read())

In [21]:
scheme = load_scheme("iris-scheme.avsc")
scheme

In [26]:
try:
    running = True
    while running:
        msg = consumer.poll(timeout=60000)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                sys.stderr.write("Reached end.")
            elif msg.error():
                raise KafkaException(msg.error())
        message = msg.value()
        bytes_reader = io.BytesIO(message)
        deserialzer = avro.io.BinaryDecoder(bytes_reader)
        reader = avro.io.DatumReader(scheme)
        try:
            decoded = reader.read(deserialzer)
            print(decoded)
            sys.stdout.flush()
        except AssertionError:
            continue
except KeyboardInterrupt:
    sys.stderr.write("Keyboard interrupt...")

Keyboard interrupt...